In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from numba import njit

: 

In [ ]:
# Please see process_L2 in https://github.com/nkaz001/data-tardis/blob/master/tardis-process.ipynb
#
# x[0] price of a level in the order book
# x[1] quantity of a level in the order book
#
# buy = sum(map(lambda x: x[1], filter(lambda x: x[0] > mid * (1 - p), bid.iteritems())))
# sell = sum(map(lambda x: x[1], filter(lambda x: x[0] < mid * (1 + p), ask.iteritems())))
# imbalance = standardize(buy - sell, 1-day)
# 
# When not considering skew, this strategy does 
# Buy if imbalance is greater than a given threshold +T
# Sell if imbalance is less than a given threshold -T

In [ ]:
df = pd.read_pickle('btcusdt_data')
df.iloc[:, 7:] = (df.iloc[:, 7:] - df.iloc[:, 7:].rolling('1d').mean()) / df.iloc[:, 7:].rolling('1d').std()  # 1-day rolling

In [ ]:
df.columns

In [ ]:
df.iloc[-25000:]

In [ ]:
@njit
def predict_njit(fee_, A, B, half_spread, max_position, X, imbalance):
    tick_size = 0.01
    running_qty = 0
    static_equity = 0
    fee = 0
    equity = []
    running_qty_ = []
    order_qty = 10000
    order_qty_btc = 0
    new_bid = np.nan
    new_ask = np.nan
    high = 1
    low = 2
    close = 3
    volume = 4
    best_bid = 5
    best_ask = 6
    for row in X:
        # Check if the orders are filled:
        # the bid order is considered filled if a trade happened below the order price.
        # the ask order is considered filled if a trade happened above the order price.
        # 
        #if np.isfinite(new_bid) \
        #    and (int(round(new_bid / tick_size)) < int(round(row[low] / tick_size)) or int(round(new_bid / tick_size) >= row[best_ask])) \
        #    and row[volume] > 0:
        if np.isfinite(new_bid) \
            and int(round(new_bid / tick_size)) > int(round(row[low] / tick_size)) \
            and row[volume] > 0:
            running_qty += order_qty_btc
            static_equity -= order_qty_btc * new_bid
            fee += order_qty_btc * new_bid * fee_    
        #if np.isfinite(new_ask) \
        #    and (int(round(new_ask / tick_size)) < int(round(row[high] / tick_size)) or int(round(new_ask / tick_size) <= row[best_bid])) \
        #    and row[volume] > 0:
        if np.isfinite(new_ask) \
            and int(round(new_ask / tick_size)) < int(round(row[high] / tick_size)) \
            and row[volume] > 0:
            running_qty -= order_qty_btc
            static_equity += order_qty_btc * new_ask
            fee += order_qty_btc * new_ask * fee_
        equity.append(static_equity + running_qty * row[close] - fee)
        running_qty_.append(running_qty)
        x = running_qty * row[close] # in $
        skew = B * x / max_position * -1
        quote_mid_price = row[close] + A * row[imbalance] + skew
        # Limit the price of orders to best bid/ask in order not to take liquidity. (Due to fee)
        new_bid = np.minimum(np.minimum(np.round(quote_mid_price * (1 - half_spread) / tick_size) * tick_size, row[close] - tick_size), row[best_bid])
        new_ask = np.maximum(np.maximum(np.round(quote_mid_price * (1 + half_spread) / tick_size) * tick_size, row[close] + tick_size), row[best_ask])
        order_qty_btc = np.maximum(np.round(order_qty / row[close], 3), 0.001)
        # The maximum position size is fixed in Dollar.
        if x > max_position:
            new_bid = np.nan
        if x < -max_position:
            new_ask = np.nan
    return equity, running_qty_

class Backtest:
    def __init__(self, fee=None, max_position=None, A=None, B=None, half_spread=None, imbalance=None):
        self.fee = fee
        self.max_position = max_position
        self.A = A
        self.B = B
        self.half_spread = half_spread
        self.imbalance = imbalance
        
    def set_params(self, A, B, half_spread, imbalance):
        self.A = A
        self.B = B
        self.half_spread = half_spread
        self.imbalance = imbalance
        return self
        
    def get_params(self, deep=True):
        return { 'fee': self.fee, 'max_position': self.max_position, 'A': self.A, 'B': self.B, 'half_spread': self.half_spread, 'imbalance': self.imbalance }
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        equity, running_qty = predict_njit(self.fee, self.A, self.B, self.half_spread, self.max_position, X, self.imbalance)
        return equity, running_qty
    
    def score(self, X):
        equity, _ = self.predict(X)
        returns = (pd.Series(equity).diff() / self.max_position).fillna(0)
#         bm_returns = pd.Series(X[:, 3]).pct_change().fillna(0)
#         returns_ = returns - bm_returns
        return np.divide(returns.mean(), returns.std())

In [ ]:
train = df[(df.index >= '2021-1-1') & (df.index < '2022-1-1')]
valid = df[df.index < '2021-1-1']

In [ ]:
param_dist = { 'A': stats.uniform(1, 100), 'B': stats.uniform(1, 100), 'half_spread': stats.uniform(0, 0.001), 'imbalance': np.arange(7, 19) }
search = RandomizedSearchCV(Backtest(0.0002, 1000000),
                            cv=[(np.arange(len(train)), np.arange(len(train)))],
                            param_distributions=param_dist,
                            verbose=1,
                            n_iter=1000,
                            n_jobs=8)
search.fit(train.values)

In [ ]:
# v2.4 params. Feb. 15, 2022
search.best_params_ = {
    'A': 34.79953363189946,
    'B': 81.73462819589255,
    'half_spread': 0.000987752995703277,
    'imbalance': 12
}
search.best_estimator_.set_params(**search.best_params_)

In [ ]:
search.best_params_

In [ ]:
search.best_estimator_.score(train.values)

In [ ]:
equity, running_qty = search.best_estimator_.predict(train.values)
equity = pd.Series(equity, index=train.index)
running_qty = pd.Series(running_qty, index=train.index)

In [ ]:
equity.plot()

In [ ]:
train['close'].plot()

In [ ]:
running_qty.plot()

In [ ]:
def calc_cagr(begin, final, years):
    if final < 0:
        return (-1) * (((abs(final) + 2 * begin) / begin) ** (1 / years) - 1)
    else:
        return (final / begin) ** (1 / years) - 1

In [ ]:
returns = equity.resample('1d').last().diff() / search.best_estimator_.max_position
# bm_returns = train['close'].resample('1d').last().pct_change()
# returns_ = returns - bm_returns
returns_ = returns
sr = np.divide(returns_.mean(), returns_.std()) * np.sqrt(365)

equity_1d = equity.resample('1d').last()
Roll_Max = equity_1d.cummax()
Daily_Drawdown = (equity_1d - Roll_Max) / search.best_estimator_.max_position
Max_Daily_Drawdown = Daily_Drawdown.cummin()

period = (equity.index[-1] - equity.index[0]).days

print(pd.Series({
    'Start date': equity.index[0].strftime('%Y-%m-%d'),
    'End date': equity.index[-1].strftime('%Y-%m-%d'),
    'Time period (days)': period,
    'Sharpe Ratio': sr,
    'CAGR': calc_cagr(search.best_estimator_.max_position, search.best_estimator_.max_position + equity[-1], period / 365),
    'Max Daily Drawdown': -Max_Daily_Drawdown.min(),
}))

returns.hist(bins=20)

In [ ]:
equity, running_qty = search.best_estimator_.predict(df.values)
equity = pd.Series(equity, index=df.index)
running_qty = pd.Series(running_qty, index=df.index)


returns = equity.resample('1d').last().diff() / search.best_estimator_.max_position
#bm_returns = valid['close'].resample('1d').last().pct_change()
#returns_ = returns - bm_returns
returns_ = returns
sr = np.divide(returns_.mean(), returns_.std()) * np.sqrt(365)

equity_1d = equity.resample('1d').last()
Roll_Max = equity_1d.cummax()
Daily_Drawdown = (equity_1d - Roll_Max) / search.best_estimator_.max_position
Max_Daily_Drawdown = Daily_Drawdown.cummin()

period = (equity.index[-1] - equity.index[0]).days

print(pd.Series({
    'Start date': equity.index[0].strftime('%Y-%m-%d'),
    'End date': equity.index[-1].strftime('%Y-%m-%d'),
    'Time period (days)': period,
    'Train period': '%s - %s' % (train.index[0].strftime('%Y-%m-%d'), train.index[-1].strftime('%Y-%m-%d')),
    'Valid period': '%s - %s' % (valid.index[0].strftime('%Y-%m-%d'), valid.index[-1].strftime('%Y-%m-%d')),
    'Sharpe ratio': sr,
    'CAGR': calc_cagr(search.best_estimator_.max_position, search.best_estimator_.max_position + equity[-1], period / 365),
    'RRR': calc_cagr(search.best_estimator_.max_position, search.best_estimator_.max_position + equity[-1], period / 365) / -Max_Daily_Drawdown.min(),
    'Maximum drawdown': -Max_Daily_Drawdown.min(),
}))

# equity.resample('1d').last().pct_change().hist(bins=20)

import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
ax1 = (equity / search.best_estimator_.max_position * 100).plot()
ax2 = ((df['close'] / df['close'][0] - 1) * 100).plot()
ax1.set_ylabel('Cumulative Returns (%)')
ax1.legend(["Strategy's equity (VIP0 maker fee)", 'Binance Futures BTCUSDT'])
ax1.grid()

In [ ]:
Daily_Drawdown.plot()

In [ ]:
(equity / search.best_estimator_.max_position).plot()

In [ ]:
equity.to_pickle("btcusdt-marketmaking-vip0")

In [ ]:
# v2.4 params. Feb. 15, 2022
search.best_params_ = {
    'A': 27.523717761230337,
    'B': 73.23408011910584,
    'half_spread': 0.0009954163968328592,
    'imbalance': 12
}
search.best_estimator_.set_params(**search.best_params_)
search.best_estimator_.fee = 0

In [ ]:
equity, running_qty = search.best_estimator_.predict(df.values)
equity = pd.Series(equity, index=df.index)
running_qty = pd.Series(running_qty, index=df.index)


returns = equity.resample('1d').last().diff() / search.best_estimator_.max_position
#bm_returns = valid['close'].resample('1d').last().pct_change()
#returns_ = returns - bm_returns
returns_ = returns
sr = np.divide(returns_.mean(), returns_.std()) * np.sqrt(365)

equity_1d = equity.resample('1d').last()
Roll_Max = equity_1d.cummax()
Daily_Drawdown = (equity_1d - Roll_Max) / search.best_estimator_.max_position
Max_Daily_Drawdown = Daily_Drawdown.cummin()

period = (equity.index[-1] - equity.index[0]).days

print(pd.Series({
    'Start date': equity.index[0].strftime('%Y-%m-%d'),
    'End date': equity.index[-1].strftime('%Y-%m-%d'),
    'Time period (days)': period,
    'Train period': '%s - %s' % (train.index[0].strftime('%Y-%m-%d'), train.index[-1].strftime('%Y-%m-%d')),
    'Valid period': '%s - %s' % (valid.index[0].strftime('%Y-%m-%d'), valid.index[-1].strftime('%Y-%m-%d')),
    'Sharpe ratio': sr,
    'CAGR': calc_cagr(search.best_estimator_.max_position, search.best_estimator_.max_position + equity[-1], period / 365),
    'RRR': calc_cagr(search.best_estimator_.max_position, search.best_estimator_.max_position + equity[-1], period / 365) / -Max_Daily_Drawdown.min(),
    'Maximum drawdown': -Max_Daily_Drawdown.min(),
}))

# equity.resample('1d').last().pct_change().hist(bins=20)

import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
ax1 = (equity / search.best_estimator_.max_position * 100).plot()
ax2 = ((df['close'] / df['close'][0] - 1) * 100).plot()
ax1.set_ylabel('Cumulative Returns (%)')
ax1.legend(["Strategy's equity (VIP9 maker fee)", 'Binance Futures BTCUSDT'])
ax1.grid()

In [ ]:
equity.to_pickle("btcusdt-marketmaking-vip9")